In [1]:
import ipywidgets as widgets
from IPython.display import display
import pandas as pd
import hashlib
import random
import os
import pickle
import joblib
os.chdir('Resources/')

In [2]:
def load_key(filename):
    with open(filename, "rb") as f:
        return pickle.load(f)

In [3]:
def string_to_int(message):
    return int.from_bytes(message.encode('utf-8'), byteorder='big')

def int_to_string(value):
    byte_length = (value.bit_length() + 7) // 8 
    return value.to_bytes(byte_length, byteorder='big').decode('utf-8', errors='ignore')

def float_to_string(message):
    return str(message)

def string_to_float(message):
    return float(message)

In [4]:
def power(b, p, m):
    b = b % m
    if p == 0:
        return 1
    j = power(b, p // 2, m)
    j = (j * j) % m
    if p % 2 == 1:
        j = (j * b) % m
    return j

In [5]:
def mod_inv(a, m):
    origin_m = m
    y, x = 0, 1

    if m == 1:
        return 0

    while a > 1:
        q = a // m
        temp = m

        m = a % m
        a = temp
        temp = y

        y = x - q * y
        x = temp

    if x < 0:
        x += origin_m
    return x

In [6]:
df = pd.read_csv('20_(SYS)_CC_Received_User_Data.csv')
df1 = df.iloc[:, :5]
df2 = df.iloc[:, 5:10]
df3 = df.iloc[:, 10:]  
df3

,ST_Slope,HeartDisease
0,Flat,NaN


In [7]:
def encrypt_data(n, df, public_keys, column_names):
    encrypted_data = pd.DataFrame()

    for k in range(n):
        public_key = public_keys[column_names[k]] 
        p = public_key['p']
        g = public_key['g']
        h = public_key['h']
        y = public_key['y']
            
        print(f"Processing Column: {column_names[k]}")

        s = power(h, y, p)

        messages = df.iloc[:, k].values 

        encrypted_column = []

        for i, message in enumerate(messages, 1):
            if message is not None and message == message:  
                print(f"Processing Message {i}: {message}")
                
                if isinstance(message, str):
                    temp_message = string_to_int(message)
                elif isinstance(message, float):
                    temp_message = string_to_int(float_to_string(message))
                else:
                    temp_message = int(message)  
                
                c1 = power(g, y, p)
                c2 = (temp_message * s) % p
                
                encrypted_column.append(f"({c1},{c2})") 
                print(f"Ciphertext: (c1, c2): ({c1}, {c2})")
            
            else:
                encrypted_column.append(None)

        encrypted_data[column_names[k]] = encrypted_column

    return encrypted_data

In [8]:
public_keys = load_key("5_1_CR1_Public_Key_Part.pkl")

column_names = [
    "Age", "Sex", "ChestPainType", "RestingBloodPressure", "Cholesterol"
]

encrypted_data_1 = encrypt_data(5, df1, public_keys, column_names)
encrypted_data_1

Processing Column: Age
Processing Message 1: 56
Ciphertext: (c1, c2): (11611711856791845453844363544193004458436652567582, 10773622686875878692856244763039934724031240962584)
Processing Column: Sex
Processing Message 1: Male
Ciphertext: (c1, c2): (11521613315689790599790221322656441619898359351715, 2964730976727955981426344307431069086843753651983)
Processing Column: ChestPainType
Processing Message 1: Non-anginal Pain
Ciphertext: (c1, c2): (5340627710438748782301236172298939673737240321334, 8373572660573689414511738173508041614498152858317)
Processing Column: RestingBloodPressure
Processing Message 1: 140
Ciphertext: (c1, c2): (2959799491122696844814186940885321444153216238962, 13356721525663435021843137376573774862783952558017)
Processing Column: Cholesterol
Processing Message 1: 250
Ciphertext: (c1, c2): (2572745932559499053547019353918953850096268849190, 5433374778066829517275318753852696644728036248185)


,Age,Sex,ChestPainType,RestingBloodPressure,Cholesterol
0,(116117118567918454538443635441930044584366525...,(115216133156897905997902213226564416198983593...,(534062771043874878230123617229893967373724032...,(295979949112269684481418694088532144415321623...,(257274593255949905354701935391895385009626884...


In [9]:
public_keys = load_key("5_2_CR2_Public_Key_Part.pkl")

column_names = [
    "FastingBloodSugar", "RestingECG", "MaximumHeartRate", "ExerciseAngina", "Oldpeak"
]

encrypted_data_2 = encrypt_data(5, df2, public_keys, column_names)
encrypted_data_2

Processing Column: FastingBloodSugar
Processing Message 1: Yes
Ciphertext: (c1, c2): (8991270131106317065828259596356481212681246291421, 15568715435446811299055523258210286707692570160693)
Processing Column: RestingECG
Processing Message 1: Left Ventricular Hypertrophy
Ciphertext: (c1, c2): (3734041290046056160613914136571085705122900972458, 17057184702817548962293299462681744792720153792848)
Processing Column: MaximumHeartRate
Processing Message 1: 150
Ciphertext: (c1, c2): (11607967323062549316964231219539938367253314058306, 7668520651373950135840832281204620228559205021292)
Processing Column: ExerciseAngina
Processing Message 1: No
Ciphertext: (c1, c2): (13647169038252931707909305178839774745227353223390, 3890046730447452713932648705085222577958075487680)
Processing Column: Oldpeak
Processing Message 1: 2
Ciphertext: (c1, c2): (2127171966273882616386704855669834874862200078566, 8140073766936942039891431619861208656708976578470)


,FastingBloodSugar,RestingECG,MaximumHeartRate,ExerciseAngina,Oldpeak
0,(899127013110631706582825959635648121268124629...,(373404129004605616061391413657108570512290097...,(116079673230625493169642312195399383672533140...,(136471690382529317079093051788397747452273532...,(212717196627388261638670485566983487486220007...


In [10]:
public_keys = load_key("5_3_CC_Public_Key_Part.pkl")

column_names = [
    "ST_Slope"
]

encrypted_data_3 = encrypt_data(1, df3, public_keys, column_names)
encrypted_data_3

Processing Column: ST_Slope
Processing Message 1: Flat
Ciphertext: (c1, c2): (9533738547640960361372917347170583969881856341594, 14348667148581208206885409276877045417652009792193)


,ST_Slope
0,(953373854764096036137291734717058396988185634...


In [11]:
encrypted_df = pd.concat([encrypted_data_1, encrypted_data_2, encrypted_data_3], axis='columns')
encrypted_df
encrypted_df.to_csv('21_(SYS)_CR_Encrypted_User_Data.csv', index=False)